In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
os.chdir('/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/PHCellTrackSeg')
!ls
import pandas as pd


drive_data_management.ipynb  Mask_merge.ipynb  res_track.txt
im2RGB.py		     README.md	       stack2im.py
man_track.txt		     requirements.txt  utils


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 36.6MB 112kB/s 
     |████████████████████████████████| 42.5MB 100kB/s 


In [4]:
import sys
from utils.build_processed_videos import get_accuracy_measures_videos, build_videos, jaccard_index, dice_coeff
import utils.build_processed_videos
import os
import SimpleITK as sitk
import numpy as np
from scipy.spatial.distance import directed_hausdorff
import sys
from skimage.measure import regionprops
from scipy import ndimage

In [5]:
def jaccard_index (y_pred, y_true):
    intersection = y_true*y_pred
    intersection = intersection.astype(np.float)
    union = y_true + y_pred - intersection
    union = union.astype(np.float)
    if np.sum(union) == 0.0:
        return 1.0
    else:
        return np.sum(intersection)/np.sum(union)

def dice_coeff (y_pred, y_true):
    intersection = y_true*y_pred
    intersection = intersection.astype(np.float)
    union = y_true + y_pred
    union = union.astype(np.float)
    if np.sum(union) == 0.0:
        return 1.0
    else:
        return 2*np.sum(intersection)/np.sum(union)

In [6]:
def get_accuracy_measures_validation(PATH2VIDEOS, PATH2VALSCORE, PATH2STORE, EXPERIMENT):
    '''
    PATH2RESULTS: path where the results are placed.
    PATH2GT: path to the ground truth data.
    FILE2STORE: path to a file where the results will be saved.
    PATH2VIDEOS: path to a txt file in which each row contains the name of the video and the frame that belongs to that
                video:
        Labels row      Videos ; Frames
                        video 1; raw_001.tif\n
                        video 1; raw_002.tif\n
                        ...
                        video 1; raw_032.tif\n
                        video 2; raw_033.tif\n
    '''

    # FILES = os.listdir(PATH2VIDEOS)

    # Read the file containing the relation between the initial videos and the individual frames.
    files = [x for x in open(PATH2VIDEOS, "r")]
    files = files[1:]  # First row contains labels
    file_relation = [[x.split(';')[0], x.split(';')[1][:-1]] for x in files]

    results = pd.read_csv(PATH2VALSCORE,sep=',',header=0)
    
#    Need to change the name of the frames to adapt to our input (diferentiate between gt and pred)

    COUNT = 1
    if not os.path.exists(PATH2STORE):
        os.makedirs(PATH2STORE)

    while COUNT <= len(file_relation):
        # Get the name of the original videos and the number of frames that it contains
        file_name = file_relation[COUNT][0]  # video name
        # Calculate how many frames you need to process (it is said in the name of the video)
        start_time, end_time = file_name.split('_')[-1].split('-')
        start_time = np.int(start_time)
        end_time = np.int(end_time)
        # TODO: check how many frames belong to the same video and reconstruct them
        sys.stdout.write("\rProcessing video {0}:\n".format(file_name))
        
        Cell_validation=np.zeros(len(range(end_time - start_time +1)))
        precision=np.zeros(len(range(end_time-start_time+1)))
        fp = np.zeros(len(range(end_time-start_time+1)))
        
        for i in range(end_time - start_time + 1):
            # Load the single frame
            frame_name = os.path.join(PATH2VIDEOS, 'raw_{0:0>3}_pred.tif'.format(int(COUNT + i)))

            # Calculate the mean accuracy measures for each video
            Cell_validation [i] = results[' Cell'][COUNT+i]
            precision [i] = results[' precision'][COUNT+i]
            fp[i] = (Cell_validation [i]/ precision [i])-Cell_validation [i]
            progress = (i + 1) / (end_time - start_time + 1)
            

            text = "\r[{0}] {1}%".format("-" * (i + 1) + " " * (end_time - start_time - i), progress * 100)
            sys.stdout.write(text)
            sys.stdout.flush()
        sys.stdout.write("\n")  # this ends the progress bar
        COUNT = COUNT + i + 1
        Cell_vali = np.mean(Cell_validation)
        prec = np.mean(precision)
        fpt = np.mean(fp)
        
        # Store the measures for each video in a csv file
        if os.path.exists(os.path.join(PATH2STORE, 'video_accuracies_val' + EXPERIMENT + '.csv')):
            with open(os.path.join(PATH2STORE, 'video_accuracies_val' + EXPERIMENT +'.csv'), mode='a') as file_:
                file_.write(file_name + ";{};{};{}".format(Cell_vali, prec, fpt))
                file_.write("\n")
        else:
            fields = "video; Cell_validation; Precission; FP;"
            with open(os.path.join(PATH2STORE, 'video_accuracies_val' + EXPERIMENT +'.csv'), 'w') as file_:
                file_.write(fields)
                file_.write("\n")
                file_.write(file_name + ";{};{};{}".format(Cell_vali,prec,fpt))
                file_.write("\n")
                
    print("All videos have been evaluated")

In [7]:
PATH2VIDEOS = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/data4training/test/videos2im_relation.csv'
PATH2VALSCORE = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/Seg_github/RESULTS-SERVER/checkpoints/constant_lr_tiramisu103_v01/2900/val_scores.csv'
PATH2STORE = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/Seg_github/eval_data_server/validation/'
EXPERIMENT = '_Model5'

In [8]:
get_accuracy_measures_validation(PATH2VIDEOS, PATH2VALSCORE, PATH2STORE, EXPERIMENT)

Processing video 5000_6-15-14_1_xy004_255-354:
[----------------------------------------------------------------------------------------------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-8_6-15-14_1_xy033_181-183:
[---] 100.0%
Processing video 10000_11-20-13_1003_xy015_002-004:
[---] 100.0%
Processing video 5000_7-1-14_001_xy001_240-242:
[---] 100.0%
Processing video 10000_11-20-13_1003_xy008_078-080:
[---] 100.0%
Processing video 5000_7-1-14_001_xy005_157-171:
[---------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-6_6-1-14_1001_xy28_330-332:
[---] 100.0%
Processing video 10000_11-20-13_1003_xy018_267-281:
[---------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-6_12-5-12_1_xy27_370-385:
[----------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-6_12-5-12_1_xy30_090-104:
[---------------] 100.0%
Processing video Videos and corresponding protrusion analy

KeyError: ignored

Get accuaracy measures

In [ ]:
PATH2RESULTS = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/Seg_github/Semantic-Segmentation-Suite/constante_tiramisu/2980/'
PATH2GT = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/Seg_github/Semantic-Segmentation-Suite/constante_tiramisu/2980/'
PATH2STORE = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/Seg_github/eval_data_server/constante_tiramisu/2980/'
PATH2VIDEOS = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/data4training/test/videos2im_relation.csv'
EXPERIMENT = '2980'

In [ ]:
get_accuracy_measures_videos(PATH2RESULTS, PATH2GT, PATH2STORE, PATH2VIDEOS, EXPERIMENT)

Processing video 5000_6-15-14_1_xy004_255-354:
[----------------------------------------------------------------------------------------------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-8_6-15-14_1_xy033_181-183:
[---] 100.0%
Processing video 10000_11-20-13_1003_xy015_002-004:
[---] 100.0%
Processing video 5000_7-1-14_001_xy001_240-242:
[---] 100.0%
Processing video 10000_11-20-13_1003_xy008_078-080:
[---] 100.0%
Processing video 5000_7-1-14_001_xy005_157-171:
[---------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-6_6-1-14_1001_xy28_330-332:
[---] 100.0%
Processing video 10000_11-20-13_1003_xy018_267-281:
[---------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-6_12-5-12_1_xy27_370-385:
[----------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-6_12-5-12_1_xy30_090-104:
[---------------] 100.0%
Processing video Videos and corresponding protrusion analy

In [ ]:
def get_accuracy_measures_videos(PATH2RESULTS, PATH2GT, PATH2STORE, PATH2VIDEOS, EXPERIMENT):
    '''
    PATH2RESULTS: path where the results are placed.
    PATH2GT: path to the ground truth data.
    FILE2STORE: path to a file where the results will be saved.
    PATH2VIDEOS: path to a txt file in which each row contains the name of the video and the frame that belongs to that
                video:
        Labels row      Videos ; Frames
                        video 1; raw_001.tif\n
                        video 1; raw_002.tif\n
                        ...
                        video 1; raw_032.tif\n
                        video 2; raw_033.tif\n
    '''

    # FILES = os.listdir(PATH2VIDEOS)

    # Read the file containing the relation between the initial videos and the individual frames.
    files = [x for x in open(PATH2VIDEOS, "r")]
    files = files[1:]  # First row contains labels
    file_relation = [[x.split(';')[0], x.split(';')[1][:-1]] for x in files]
    
#    Need to change the name of the frames to adapt to our input (diferentiate between gt and pred)

    COUNT = 1
    if not os.path.exists(PATH2STORE):
        os.makedirs(PATH2STORE)

    while COUNT <= len(file_relation):
        # Get the name of the original videos and the number of frames that it contains
        file_name = file_relation[COUNT][0]  # video name
        # Calculate how many frames you need to process (it is said in the name of the video)
        start_time, end_time = file_name.split('_')[-1].split('-')
        start_time = np.int(start_time)
        end_time = np.int(end_time)
        # TODO: check how many frames belong to the same video and reconstruct them
        sys.stdout.write("\rProcessing video {0}:\n".format(file_name))
        HM=np.zeros(len(range(end_time - start_time +1)))
        JAC = np.zeros(len(range(end_time - start_time +1)))
        DICE = np.zeros(len(range(end_time - start_time +1)))
        DICE_cell_all=np.zeros(len(range(end_time - start_time +1)))
        JAC_cell_all=np.zeros(len(range(end_time - start_time +1)))
        HM_cell_all = np.zeros(len(range(end_time - start_time +1)))
        for i in range(end_time - start_time + 1):
            # Load the single frame
            frame_name = os.path.join(PATH2RESULTS, 'raw_{0:0>3}_pred.tif'.format(int(COUNT + i)))
            frame = sitk.ReadImage(frame_name)
            frame = sitk.GetArrayFromImage(frame)
            gt_name = os.path.join(PATH2GT, 'raw_{0:0>3}_gt.tif'.format(int(COUNT + i)))
            gt = sitk.ReadImage(gt_name)
            gt = sitk.GetArrayFromImage(gt)
            gt[gt > 0] = 1
            frame[frame < 0.5] = 0
            frame[frame > 0] = 1
            # Calculate the mean accuracy measures for each video
            HM[i]=directed_hausdorff(frame,gt)[0]
            JAC[i] = jaccard_index(frame.flatten(), gt.flatten())
            DICE [i] = dice_coeff(frame.flatten(), gt.flatten())
            DICE_cell_all[i], JAC_cell_all[i], HM_cell_all[i] = get_accuracy_cells(gt,frame)
            
            # if i == 0:
            #     HM[i] = directed_hausdorff(frame, gt)[0]
            #     # HM_total = HM

            #     JAC = jaccard_index(frame.flatten(), gt.flatten())
            #     # JAC_total = JAC

            #     DICE = dice_coeff(frame.flatten(), gt.flatten())
            #     # DICE_total = DICE

            #     DICE_cell_all, JAC_cell_all, HM_cell_all = get_accuracy_cells(gt,frame)
            # else:
            #     HM = (HM*i + directed_hausdorff(frame, gt)[0])/(i+1)
            #     # HM_total = directed_hausdorff(frame, gt)[0]+HM_total

            #     JAC = (JAC*i + jaccard_index(frame.flatten(), gt.flatten()))/(i+1)
            #     # JAC_total = jaccard_index(frame.flatten(), gt.flatten()) + JAC_total

            #     DICE = (DICE*i + dice_coeff(frame.flatten(), gt.flatten()))/(i+1)
            #     # DICE_total = dice_coeff(frame.flatten(), gt.flatten()) + DICE_total
                
                
            #     DICE_cell, JAC_cell, HM_cell = get_accuracy_cells(gt,frame)
            #     DICE_cell_all = (DICE_cell_all*i + DICE_cell)/(i+1)
            #     JAC_cell_all = (JAC_cell_all*i + JAC_cell)/(i+1)
            #     HM_cell_all = (HM_cell_all*i + HM_cell)/(i+1)


            progress = (i + 1) / (end_time - start_time + 1)
            text = "\r[{0}] {1}%".format("-" * (i + 1) + " " * (end_time - start_time - i), progress * 100)
            sys.stdout.write(text)
            sys.stdout.flush()
        sys.stdout.write("\n")  # this ends the progress bar
        COUNT = COUNT + i + 1
        HM_mean = np.mean(HM)
        HM_dev = np.std(HM)
        JAC_mean = np.mean(JAC)
        JAC_dev = np.std(JAC)
        DICE_mean = np.mean(DICE)
        DICE_dev = np.std(DICE)
        DICE_cell_all_mean = np.mean(DICE_cell_all)
        DICE_cell_all_dev = np.std(DICE_cell_all)
        JAC_cell_all_mean = np.mean(JAC_cell_all)
        JAC_cell_all_dev = np.std(JAC_cell_all)
        HM_cell_all_mean = np.mean(HM_cell_all)
        HM_cell_all_dev = np.std(HM_cell_all)
                
        # Store the measures for each video in a csv file
        if os.path.exists(os.path.join(PATH2STORE, 'video_accuracies_' + EXPERIMENT + '.csv')):
            with open(os.path.join(PATH2STORE, 'video_accuracies_' + EXPERIMENT +'.csv'), mode='a') as file_:
                file_.write(file_name + ";{};{};{};{};{};{};{};{};{};{};{};{}".format(HM_mean, HM_dev, JAC_mean, JAC_dev, DICE_mean, DICE_dev, HM_cell_all_mean, HM_cell_all_dev, JAC_cell_all_mean, JAC_cell_all_dev, DICE_cell_all_mean, DICE_cell_all_dev))
                # file_.write(file_name + ";{};{};{}".format(HM, JAC, DICE))
                # for i in range(total_cells):
                #     file_.write( ";{};{};{};".format(HM_cell[i], JAC_cell[i] , DICE_cell[i]))
                file_.write("\n")
        else:
            fields = "video; Hausdorff-distance; Haus_dev; Jaccard index; JAC_ dev; Dice coeffficient; DICE_dev; Mean Hausdorff-distance; Mean_Haus_dev; Mean Jaccard index; Mean_JAC_dev; Mean Dice coefficient; Mean_DICE_dev"
            # fields = "video; Hausdorff-distance; Jaccard index; Dice coeffficient"
            with open(os.path.join(PATH2STORE, 'video_accuracies_' + EXPERIMENT +'.csv'), 'w') as file_:
                file_.write(fields)
                file_.write("\n")
                file_.write(file_name + ";{};{};{};{};{};{};{};{};{};{};{};{}".format(HM_mean, HM_dev, JAC_mean, JAC_dev, DICE_mean, DICE_dev, HM_cell_all_mean, HM_cell_all_dev, JAC_cell_all_mean, JAC_cell_all_dev, DICE_cell_all_mean, DICE_cell_all_dev))
                # file_.write(file_name + ";{};{};{}".format(HM, JAC, DICE))
                # for i in range(total_cells):
                #     file_.write( ";{};{};{};".format(HM_cell[i], JAC_cell[i] , DICE_cell[i]))
                file_.write("\n")
                
    print("All videos have been evaluated")
    
  
    
def get_accuracy_cells (gt, pred):
   
    gt_labels, gt_nlabels = ndimage.measurements.label(gt)
    pred_labels, pred_nlabels = ndimage.measurements.label(pred)

    HM= np.zeros(pred_nlabels)
    JAC = np.zeros(pred_nlabels)
    DICE= np.zeros(pred_nlabels)
    count=0
    val_nlabels = np.arange(gt_nlabels+1)
    

#Esto cuenta con respecto de las células que coinciden con el ground truth, pero como el jac y el dice están hechos a la length de la predicción, si hay células que no estan, el jac y el dice ya son cero de por si
    for j in val_nlabels[1:]:
      gt_label = gt_labels == int(j)
      gt_label [gt_label>0.5] =1
      overlap = np.multiply(gt_label, pred_labels)
      if np.sum(overlap)>0:
            candidates = np.unique(overlap) # at least two values: 0 and prediction_label
            overlap_sum = 0 #creo que inutil
            for i in candidates[1:]:
                # label2eval = 0
                # overlap_sum = np.sum(overlap == int(i))
                # if np.sum(overlap == int(i)) > overlap_sum and np.sum(overlap == int(i)) >= 0.5*np.sum(gt_label) :
                if np.sum(overlap == int(i)) >= 0.5*np.sum(gt_label) :
                    label2eval = int(i)
                    pred2eval =[]
                    pred2eval=pred_labels==int(i)
                    pred2eval=1*pred2eval
                    HM [count] = directed_hausdorff(pred2eval, gt_label)[0]
                    JAC [count] = jaccard_index(pred2eval.flatten(), gt_label.flatten())
                    DICE [count] = dice_coeff(pred2eval.flatten(), gt_label.flatten())
                    pred_labels[pred_labels == int(i)] = 0 # creo que inutil 
                    count +=1

    HM = np.sum(HM)/pred_nlabels
    JAC = np.sum(JAC)/pred_nlabels
    DICE = np.sum(DICE)/pred_nlabels
   
    return DICE, JAC, HM

In [ ]:
INPUTPATH = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/Seg_github/Semantic-Segmentation-Suite/Test/'
OUTPUTPATH = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/intento/pred/'
PATH2VIDEOS = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/data4training/test/videos2im_relation.csv'

In [ ]:
def build_videos(INPUTPATH,OUTPUTPATH,PATH2VIDEOS):
    '''
    INPUTPATH: path where the single frames are placed.
    OUTPUTPATH: pathe where the reconstructed videos will be stored.
    PATH2VIDEOS: path to a txt file in which each row contains the name of the video and the frame that belongs to that
                video:
        Labels row      Videos ; Frames
                        video 1; raw_001.tif\n
                        video 1; raw_002.tif\n
                        ...
                        video 1; raw_032.tif\n
                        video 2; raw_033.tif\n
    '''
    files = [x for x in open(PATH2VIDEOS, "r")]
    files = files[1:]  # First row contains labels
    file_relation = [[x.split(';')[0], x.split(';')[1][:-1]] for x in files]

    if not os.path.exists(OUTPUTPATH):
        os.makedirs(OUTPUTPATH)

    COUNT = 1
    while COUNT <= len(file_relation):
        # Get the name of the original videos and the number of frames that it contains
        file_name = file_relation[COUNT][0] # video name
        # Calculate how many frames you need to process (it is said in the name of the video)
        start_time, end_time = file_name.split('_')[-1].split('-')
        start_time = np.int(start_time)
        end_time = np.int(end_time)
        sys.stdout.write("\rProcessing video {0}:\n".format(file_name))

        for i in range(end_time-start_time+1):
            # Load the single frame
            frame_name = os.path.join(INPUTPATH, 'raw_{0:0>3}_pred.tif'.format(int(COUNT+i)))
            frame = sitk.ReadImage(frame_name)
            frame = sitk.GetArrayFromImage(frame)
            if i == 0:
                video = frame.reshape((1,frame.shape[0], frame.shape[1]))
            else:
                video = np.concatenate((video,frame.reshape((1,frame.shape[0], frame.shape[1]))), axis=0)
            progress = (i+1)/(end_time-start_time+1)
            text = "\r[{0}] {1}%".format("-" * (i+1) + " " * (end_time-start_time-i), progress * 100)
            sys.stdout.write(text)
            sys.stdout.flush()
        COUNT = COUNT + i + 1
        sitk.WriteImage(sitk.GetImageFromArray(video.astype(np.float32)), os.path.join(OUTPUTPATH, file_name + '.tif'))
        sys.stdout.write("\n")  # this ends the progress bar
    print("All videos have been reconstructed")

In [ ]:
build_videos(INPUTPATH,OUTPUTPATH,PATH2VIDEOS)

Processing video 5000_6-15-14_1_xy004_255-354:
[----------------------------------------------------------------------------------------------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-8_6-15-14_1_xy033_181-183:
[---] 100.0%
Processing video 10000_11-20-13_1003_xy015_002-004:
[---] 100.0%
Processing video 5000_7-1-14_001_xy001_240-242:
[---] 100.0%
Processing video 10000_11-20-13_1003_xy008_078-080:
[---] 100.0%
Processing video 5000_7-1-14_001_xy005_157-171:
[---------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-6_6-1-14_1001_xy28_330-332:
[---] 100.0%
Processing video 10000_11-20-13_1003_xy018_267-281:
[---------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-6_12-5-12_1_xy27_370-385:
[----------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-6_12-5-12_1_xy30_090-104:
[---------------] 100.0%
Processing video Videos and corresponding protrusion analy